# GRUs
- They're the next innovation after LSTMs but before Transformers.
- They are an adaptation of RNNs, similar to LSTMs
- They have less tensor operators so they are speedier to train. 
- You can think of them as LSTMs without the short term memory, but they do more operations to the long term memory.
- Researchers try both LSTMs and GRUs to see what works better for their usecase.